## Ejemplo de una Central Real

Caso ejemplo de la [central hidroeléctrica de Aldedávila](https://es.wikipedia.org/wiki/Presa_de_Aldead%C3%A1vila), sobre el rio Duero. 

Salto bruto en la central $H' = 140\, m$ consumo aproximado es de $615\, m^3/s$, la potencia mecánica total $750 \,MW$, el número de turbinas $n_{turb} = 6$. Los alternadores tienen $Z = 16$ pares de polos y la potencia eléctrica de cada uno de ellos es de $120\,MW$, para una  frecuencia de generación de  $50\,Hz$. Las tuberias forzadas tienen un diámetro de  $D = 5\, m$ con una longitud de $184\, m$ y se puede considerar un coeficiente de fricción $f = 0.04$. Las pérdidas y accidentes de flujo pueden considerarse con una longitud equivalente $L_{eq} = 55 \,m$.

Calcular:
- Salto neto
- Potencia hidráulica
- Rendimeinto de la instalación
- Rendimiento de la generación
- Calcular el número de velocidad específica de las turbinas, $n_s$
- Tipo de turbina se adapta para las condiciones de instalación

In [16]:
from scipy.constants import g
from math import *

In [17]:
HBruto = 140 # m, H'
Q = 615 # m^3/s
PowMec = 750 # Mw
PowGen = 120 # MW

In [18]:
DTub = 5 # m
LTub = 184 # m
f = 0.04 
Leq = 55 # m
rhoAgua = 1000 # kg/m^3
nuAgua = 1e-6 # viscosidad cinemática, m^2/s

In [19]:
Nturb = 6 # número de grupos generadores

In [20]:
STub = pi / 4 * DTub**2 # Sección de las tuberías de presión
print('S = %.2f m^2' % STub)

S = 19.63 m^2


In [23]:
QTurb = Q / Nturb # Consumo de agua por turbina
print('Qturb = %.2f m^3/s' % QTurb)

Qturb = 102.50 m^3/s


In [25]:
VTub = QTurb / STub # Velocidad del flujo por las tuberías de presión
print('Vtub = %.2f m/s' % VTub)

Vtub = 5.22 m/s


#### Pérdidas en la alimentación 

El esquema muestra detalles de las pérdidas de cabezas piezométricas.

<!--![Perdidas de Instalación](./esquemaCentral.png)

<img src="./esquemaCentral.png" width="400" height="200" />-->

<img src="./esquemaCentral.png" alt="drawing" width="650"/>

Las pérdidas de carga en la tubería de presión se calculan a través de la ecuación de Darcy-Weisbach 

$$h_l = f \frac{L}{D} \frac{V^2}{2 g}$$

In [26]:
hTub = f * LTub / DTub * VTub**2 / (2*g)
print('htub = %.2f m' % hTub)

htub = 2.05 m


Las pérdidas totales son la suma de las pricipales y las secundarias. 

In [47]:
hSec = f * Leq / DTub * VTub**2 / (2*g)
hTot = hTub + hSec # h4 + h6
print('hTot = %.2f m' % hTot)

hTot = 2.66 m


El salto de turbina $H''$

In [29]:
Hn = HBruto - hTot
print('Hn = %.2f m' % Hn)

Hn = 137.34 m


La potencia hidráulica aprovechada en cada una de las seis turbinas,

In [30]:
PowHidroTurb = rhoAgua * g * Hn * Qturb / 1e6
print('PowHidroTurb = %.2f MW' % PowHidroTurb)

PowHidroTurb = 138.06 MW


El rendimiento mecánico de cada turbina,  

In [31]:
etaMec = PowMec / Nturb / PowHidroTurb
print('etaTurb = %.2f %%' % (etaMec * 100))

etaTurb = 90.54 %


El rendimiento de la generación, 

In [32]:
etaGen = PowGen / (PowMec / Nturb )
print('etagen = %.2f %%' % (etaGen * 100))

etagen = 96.00 %


El rendimiento de la instalación es el producto de los rendimientos hidráulico y el eléctrico.

In [33]:
etaInst = etaMec * etaGen
print('etaInst = %.2f %%' % (etaMec * etaGen * 100))

etaInst = 86.92 %


### Número de velocidad específica de las turbinas

Número de revoluciones de las máquinas condicionado por las de los generadores y la frecuencia de red. El número de vieultas $$N_t = \frac{60 f}{Z}$$ siendo $Z$ el número de pares de polos, $f$ la frecuencia de la red y $N_t$ las revoluciones del eje de turbina (con acoplamiento directo al generador) expresado en RPM.

El tipo de turbina se define a través de la velocidad específica $n_s$ y el salto útil $H$, y la relación es,

$$n_s = n_g \frac{\sqrt{N_t}}{H \sqrt[4]{H}}\tag{1}$$ 

estando $P_t$ en $CV$, $H$ en metros y $n_g$ en RPM.

También se puede obtener a partir del reemplazo de $N_t = \rho g H' Q$ , $\rho g \approx 10^4$ , $CV \approx 0.75 kW$

$$n_s = 3.65 \,n_g \frac{\sqrt{Q}}{H^{3/4}}\tag{2}$$ 



In [39]:
Z = 16 # Número de pares de polos del generador
fRed = 50 # Hz
Nturb = 60 * fRed / Z
print('n_gen = %.1f rpm' % Nturb)
nSpec1 = Nturb * sqrt(PowHidroTurb * 1e3 / 0.750 ) / (Hn * pow(Hn,1/4))
print('Expresión 1 : n_s = %.1f rpm' % nSpec1)
nSpec2 = 3.65 * Nturb * sqrt(Qturb ) / (pow(Hn,0.75))
print('Expresión 2 : n_s = %.1f rpm' % nSpec2)

n_gen = 187.5 rpm
Expresión 1 : n_s = 171.1 rpm
Expresión 2 : n_s = 172.7 rpm


Del diagrama de selección encontramos que corresponde con una tipo Francis.

- Pelton con 1 inyector, 5 < ns < 30
- Pelton con 2 o más inyectores, 30 < ns < 50
- Francis lenta, 50 < ns < 100
- Francis normal, 100 < ns < 200
- Francis rápida, 200 < ns < 400
- Kaplan, 500 < ns < 1000

In [42]:
def turbineClass(ns):
    if ns < 30 :
        return "Pelton (1 chorro)"
    if ns < 50 :
        return "Pelton (2 o más chorro)"
    if ns < 100 :
        return "Francis lenta"
    if ns < 200 :
        return "Francis normal"
    if ns < 400 :
        return "Francis rápida"
    if ns < 1000 :
        return "Kaplan"
    else:
        return "No clasificada"
    

In [46]:
print('Tipo de turbina = %s' %  turbineClass(nSpec1))

Tipo de turbina = Francis normal


![Velocidades Específicas en función del salto H](./grafico-Velocidad-Especifica.png)